In [1]:
## Importing Dependencies

# Standard libraries
import pandas as pd
import numpy as np
import os
import csv

# Preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Models
from sklearn.linear_model import Lasso
from sklearn.svm import SVR, SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Dimensionality Reduction
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector

# Metrics
from sklearn.metrics import matthews_corrcoef, mean_squared_error, accuracy_score, make_scorer

# Model Persistence
from joblib import dump, load

# Plotter
import matplotlib.pyplot as plt

# Argument Parser
import argparse

# Write to a log file
import logging
import sys

path = os.getcwd()

In [2]:
## Create the logger
def log_files(logname):
    
    # Instantiate the logger and set the formatting and minimum level to DEBUG.
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)
    formatter = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')

    # Display the logs in the output
    stdout_handler = logging.StreamHandler(sys.stdout)
    stdout_handler.setLevel(logging.DEBUG)
    stdout_handler.setFormatter(formatter)

    # Write the logs to a file
    file_handler = logging.FileHandler(logname)
    file_handler.setLevel(logging.INFO)
    file_handler.setFormatter(formatter)

    # Adding the file and output handlers to the logger.
    logger.addHandler(file_handler)
    #logger.addHandler(stdout_handler)

    return logger

logger = log_files('testbook.log')

## Logarithmically scalling the values.
def rescale(array=np.array(0), destination_interval=(-5,5)):

    # Rescaling the values and saving the initial range.
    array = np.log(array)
    saved_range = (array.min(), array.max())
    array = np.interp(array, saved_range, destination_interval)

    return array, saved_range

## Inverse of the rescale function to rescale the outputs.
def unscale(array, destination_interval, source_interval=(-5,5)):

    # Undoing the previous rescaling.
    array = np.interp(array, source_interval, destination_interval)
    array = np.exp(array)

    return array

## Import the dataset
def import_data(threshold):

    # Importing the full KI set into a dataframe.
    path = os.getcwd()
    df = pd.read_csv(path + '/PositivePeptide_Ki.csv')
    logger.debug('The full dataset has %i examples.' %(len(df)))

    # Rescaling the dataframe in the log10 (-5,5) range.
    df['KI (nM) rescaled'], base_range  = rescale(df['KI (nM)'], destination_interval=(-5,5))

    # Creates a column in our dataframe to classify into 3 separate buckets.  A 'small' and 'large' bucket
    # based on the threshold, and a 'do not measure bucket' for anything with a KI value of > 4000    
    df['Bucket'] = pd.cut(x=df['KI (nM)'], bins=(0, threshold, 4000, float('inf')), labels=(0,1,2))

    return df, base_range

def hyperparameter_optimizer(x, y, params, model=SVC()):

    # Use GridsearchCV to get the optimized parameters.
    logger.debug('GridSearchCV Starting')
    clf = GridSearchCV(model, params, scoring=make_scorer(matthews_corrcoef), cv=5, n_jobs=-1)
    clf.fit(x,y)

    # Showing the best paramets found on the development set.
    logger.info('Best parameter set: %s\n' %(clf.best_params_))

    # Testing on the development set.
    #logger.debug('Grid scores on development set:')
    #logger.debug('')
    #means = clf.cv_results_['mean_test_score']
    #stds = clf.cv_results_['std_test_score']
    #for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    #    logger.debug('%0.3f (+/-%0.03f) for %r' % (mean, std*2, params))
    logger.debug('GridSearchCV Finished')

    # Save the best parameters.
    bestvals = clf.best_params_

    return bestvals

def postprocessing(df, x, y, params, model=SVC()):

    # Train our model
    seeds = [33, 42, 55, 68, 74]
    i = 0

    # Initialize the sums of the acc/mcc's.
    train_accuracy_sum = 0
    train_mcc_sum = 0
    valid_accuracy_sum = 0
    valid_mcc_sum = 0

    # Optimize the hyperparameters.
    optimized_features = hyperparameter_optimizer(x, y, params, model)
    model.set_params(**optimized_features)

    for seed in seeds:
        i += 1
        logger.debug('Training:')
        x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=seed)
        model.fit(x_train, y_train)
        logger.debug('Training Finished.')

        # Test the model on the training set.
        y_train_pred = model.predict(x_train)
        train_accuracy = accuracy_score(y_train, y_train_pred)
        train_mcc = matthews_corrcoef(y_train, y_train_pred)

        # Test the model on the validation set.
        y_valid_pred = model.predict(x_valid)
        valid_accuracy = accuracy_score(y_valid, y_valid_pred)
        valid_mcc = matthews_corrcoef(y_valid, y_valid_pred)

        # Log the individual folds
        logger.info('Training Accuracy: %3.3f, Training MCC: %3.3f, Validation Accuracy: %3.3f, '
                    'Validation MCC: %3.3f, Fold: %i\n'
                    %(train_accuracy, train_mcc, valid_accuracy, valid_mcc, i))

        # Save the results into a dataframe and display them in the logger file.
        trial = pd.DataFrame()
        trial['y_valid'] = y_valid
        trial['y_valid_pred'] = y_valid_pred
        trial['KI (nM)'] = df['KI (nM)'][trial.index]
        logger.info('Actual: | Predicted: | KI (nM)')
        for valid, pred, ki in zip(trial['y_valid'], trial['y_valid_pred'], trial['KI (nM)']):
            logger.info(' %i      |  %i         | %f' %(valid, pred, ki))
        logger.info('Fold %i finished:\n' %(i))

        # Add to the sums
        train_accuracy_sum += train_accuracy
        train_mcc_sum += train_mcc
        valid_accuracy_sum += valid_accuracy
        valid_mcc_sum += valid_mcc

        # Save the validation set with predicted and actual results.
    
    # Calculate the averages
    train_accuracy_avg = train_accuracy_sum/5
    train_mcc_avg = train_mcc_sum/5
    valid_accuracy_avg = valid_accuracy_sum/5
    valid_mcc_avg = valid_mcc_sum/5

    # Log the average scores for all the folds
    logger.info('AVG Training Accuracy: %3.3f, AVG Training MCC: %3.3f, AVG Validation Accuracy: %3.3f, '
                'AVG Validation MCC: %3.3f\n' %(train_accuracy_avg, train_mcc_avg, valid_accuracy_avg, valid_mcc_avg))



In [3]:
def sfs_pca(threshold):
    # Import the data and create our logger.
    df, _ = import_data(threshold)

    ## Model Loading for Forward Selection and PCA
    # SVC w/RBF Kernel
    sfs_rbf = load(path + '/sfs-pca/SVC with RBF Kernel %2.2f fs.joblib' %(threshold))
    pca_rbf = load(path + '/sfs-pca/SVC with RBF Kernel %2.2f pca.joblib' %(threshold))

    #XGBoost Classifier
    sfs_xgb = load(path + '/sfs-pca/XGBoost Classifier %2.2f fs.joblib' %(threshold))
    pca_xgb = load(path + '/sfs-pca/XGBoost Classifier %2.2f pca.joblib' %(threshold))

    #Random Forest Classifier
    sfs_rf = load(path + '/sfs-pca/Random Forest Classifier %2.2f fs.joblib' %(threshold))
    pca_rf = load(path + '/sfs-pca/Random Forest Classifier %2.2f pca.joblib' %(threshold))

    # Put the various elements in arrays.
    sfs_models = [sfs_rbf, sfs_xgb, sfs_rf]
    pca_models = [pca_rbf, pca_xgb, pca_rf]
    models = [SVC(kernel='rbf'), XGBClassifier(), RandomForestClassifier()]
    names = ['SVC w/RBF Kernel', 'XGBoost Classifier', 'Random Forest Classifier']

    # Set our parameters and put them in an array
    rbf_params = {'gamma': [1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
    xgb_params = {'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4]}
    rf_params = {'class_weight': ['balanced'], 'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4], 
                    'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4]}
    params_list = [rbf_params, xgb_params, rf_params]

    # Zip them up, and then proceed with model analysis.'
    logger.info('sfs-pca pipeline:\n')
    for sfs, pca, model, params, name in zip(sfs_models, pca_models, models, params_list, names):

        # Reset the variables
        x = df[df.columns[1:573]]
        y = df['Bucket']

        # Apply the transformations
        x = sfs.transform(x)
        x = pca.transform(x)

        # Train the and display results of the gridsearchcv
        logger.info('Results for %s with a threshold set to %i\n' %(name, threshold))
        postprocessing(df, x, y, params=params, model=model)

sfs_pca(threshold=0.01)
sfs_pca(threshold=10)

c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(i

In [4]:
def pca_sfs(threshold):
    # Import the data and create our logger.
    df, _ = import_data(threshold)

    ## Model Loading for Forward Selection and PCA
    # SVC w/RBF Kernel
    pca_rbf = load(path + '/pca-sfs/SVC with RBF Kernel %2.2f pca.joblib' %(threshold))
    sfs_rbf = load(path + '/pca-sfs/SVC with RBF Kernel %2.2f fs.joblib' %(threshold))

    #XGBoost Classifier
    pca_xgb = load(path + '/pca-sfs/XGBoost Classifier %2.2f pca.joblib' %(threshold))
    sfs_xgb = load(path + '/pca-sfs/XGBoost Classifier %2.2f fs.joblib' %(threshold))

    #Random Forest Classifier
    pca_rf = load(path + '/pca-sfs/Random Forest Classifier %2.2f pca.joblib' %(threshold))
    sfs_rf = load(path + '/pca-sfs/Random Forest Classifier %2.2f fs.joblib' %(threshold))

    # Put the various elements in arrays.
    pca_models = [pca_rbf, pca_xgb, pca_rf]
    sfs_models = [sfs_rbf, sfs_xgb, sfs_rf]
    models = [SVC(kernel='rbf'), XGBClassifier(), RandomForestClassifier()]
    names = ['SVC w/RBF Kernel', 'XGBoost Classifier', 'Random Forest Classifier']

    # Set our parameters and put them in an array
    rbf_params = {'gamma': [1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
    xgb_params = {'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4]}
    rf_params = {'class_weight': ['balanced'], 'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4], 
                    'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4]}
    params_list = [rbf_params, xgb_params, rf_params]

    # Zip them up, and then proceed with model analysis.
    logger.info('pca-sfs pipeline:\n')
    for sfs, pca, model, params, name in zip(sfs_models, pca_models, models, params_list, names):

        # Reset the variables
        x = df[df.columns[1:573]]
        y = df['Bucket']

        # Apply the transformations
        x = pca.transform(x)
        x = sfs.transform(x)
        
        # Train the and display results of the gridsearchcv
        logger.info('Results for %s with a threshold set to %i\n' %(name, threshold))
        postprocessing(df, x, y, params=params, model=model)

pca_sfs(threshold=0.01)
pca_sfs(threshold=10)

c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(i

In [5]:
def mms_sfs_pca(threshold):
    # Import the data and create our logger.
    df, _ = import_data(threshold)

    ## Model Loading for Forward Selection and PCA
    # SVC w/RBF Kernel
    sfs_rbf = load(path + '/mms-sfs-pca/SVC with RBF Kernel %2.2f fs.joblib' %(threshold))
    pca_rbf = load(path + '/mms-sfs-pca/SVC with RBF Kernel %2.2f pca.joblib' %(threshold))

    #XGBoost Classifier
    sfs_xgb = load(path + '/mms-sfs-pca/XGBoost Classifier %2.2f fs.joblib' %(threshold))
    pca_xgb = load(path + '/mms-sfs-pca/XGBoost Classifier %2.2f pca.joblib' %(threshold))

    #Random Forest Classifier
    sfs_rf = load(path + '/mms-sfs-pca/Random Forest Classifier %2.2f fs.joblib' %(threshold))
    pca_rf = load(path + '/mms-sfs-pca/Random Forest Classifier %2.2f pca.joblib' %(threshold))

    # Put the various elements in arrays.
    sfs_models = [sfs_rbf, sfs_xgb, sfs_rf]
    pca_models = [pca_rbf, pca_xgb, pca_rf]
    models = [SVC(kernel='rbf'), XGBClassifier(), RandomForestClassifier()]
    names = ['SVC w/RBF Kernel', 'XGBoost Classifier', 'Random Forest Classifier']

    # Set our parameters and put them in an array
    rbf_params = {'gamma': [1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
    xgb_params = {'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4]}
    rf_params = {'class_weight': ['balanced'], 'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4], 
                    'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4]}
    params_list = [rbf_params, xgb_params, rf_params]

    # Zip them up, and then proceed with model analysis.
    logger.info('mms-sfs-pca pipeline:\n')
    for sfs, pca, model, params, name in zip(sfs_models, pca_models, models, params_list, names):

        # Reset the variables
        x = df[df.columns[1:573]]
        y = df['Bucket']

        # Apply the transformations
        scaler = MinMaxScaler()
        scaler.fit(x,y)
        x = scaler.transform(x)
        x = sfs.transform(x)
        x = pca.transform(x)

        # Train the and display results of the gridsearchcv
        logger.info('Results for %s with a threshold set to %i\n' %(name, threshold))
        postprocessing(df, x, y, params=params, model=model)

mms_sfs_pca(threshold=0.01)
mms_sfs_pca(threshold=10)

c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(i

In [6]:
def mms_pca_sfs(threshold):
    # Import the data and create our logger.
    df, _ = import_data(threshold)

    ## Model Loading for Forward Selection and PCA
    # SVC w/RBF Kernel
    pca_rbf = load(path + '/mms-pca-sfs/SVC with RBF Kernel %2.2f pca.joblib' %(threshold))
    sfs_rbf = load(path + '/mms-pca-sfs/SVC with RBF Kernel %2.2f fs.joblib' %(threshold))

    #XGBoost Classifier
    pca_xgb = load(path + '/mms-pca-sfs/XGBoost Classifier %2.2f pca.joblib' %(threshold))
    sfs_xgb = load(path + '/mms-pca-sfs/XGBoost Classifier %2.2f fs.joblib' %(threshold))

    #Random Forest Classifier
    pca_rf = load(path + '/mms-pca-sfs/Random Forest Classifier %2.2f pca.joblib' %(threshold))
    sfs_rf = load(path + '/mms-pca-sfs/Random Forest Classifier %2.2f fs.joblib' %(threshold))

    # Put the various elements in arrays.
    pca_models = [pca_rbf, pca_xgb, pca_rf]
    sfs_models = [sfs_rbf, sfs_xgb, sfs_rf]
    models = [SVC(kernel='rbf'), XGBClassifier(), RandomForestClassifier()]
    names = ['SVC w/RBF Kernel', 'XGBoost Classifier', 'Random Forest Classifier']

    # Set our parameters and put them in an array
    rbf_params = {'gamma': [1e-2, 1e-3, 1e-4], 'C': [1, 10, 100, 1000]}
    xgb_params = {'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4]}
    rf_params = {'class_weight': ['balanced'], 'n_estimators': [50, 100, 150], 'max_depth': [1, 2, 3, 4], 
                    'min_samples_leaf': [1, 2, 3], 'min_samples_split': [1, 2, 3, 4]}
    params_list = [rbf_params, xgb_params, rf_params]

    # Zip them up, and then proceed with model analysis.
    logger.info('mms-pca-sfs pipeline:\n')
    for sfs, pca, model, params, name in zip(sfs_models, pca_models, models, params_list, names):

        # Reset the variables
        x = df[df.columns[1:573]]
        y = df['Bucket']

        # Apply the transformations
        scaler = MinMaxScaler()
        scaler.fit(x,y)
        x = scaler.transform(x)
        x = pca.transform(x)
        x = sfs.transform(x)
        
        # Train the and display results of the gridsearchcv
        logger.info('Results for %s with a threshold set to %i\n' %(name, threshold))
        postprocessing(df, x, y, params=params, model=model)

mms_pca_sfs(threshold=0.01)
mms_pca_sfs(threshold=10)

c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
180 fits failed out of a total of 720.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
180 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "c:\Users\a1351\.conda\envs\antithrombin\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(i